**Alfonso Ghislieri - Dec 2022 - alfonso.ghis@hotmail.fr** 

*Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*

# Covid-19 Dashboard

This dashboard uses the Public Health England (PHE) data for the covid 19 pandemic in the United Kingdom to display some comparative graphs, and allow users to interact with these graphs. The source of this data comes from PHE's open data [API](https://coronavirus.data.gov.uk/details/developers-guide/main-api).

This dashboard aims to compare:
- Daily hospital admissions with occupied mechanical ventilator beds and daily deaths.
- The number of vaccines administered daily with the number of new daily cases.

Note: some data, particularly more recent data, is missing and may cause some inaccuracies.

In [24]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import time
from uk_covid19 import Cov19API

In [25]:
# Parameters related to graph matplotlib

%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100
# make figures wider
plt.rcParams["figure.figsize"] = (10,5)


In [26]:
def load_json_data(filename):
    """ Load json files and returns rawdata """
    with open(filename+'.json', 'rt') as INFILE:
        data=json.load(INFILE)
        return data['data']

# Load raw data from json file
covid_jsondata=load_json_data('covid_data')

In [27]:
# Helper functions that are used for data wrangling

def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

def initialize_dataframe(rawdata, column_array):
    """ Set up dataframe with dates as indices and add other initial columns """
    # Create array of all dates and sort them 
    dates=[dictionary['Date'] for dictionary in rawdata]
    dates.sort()    
    
    # Create range of dates for dataframe index
    index=pd.date_range(parse_date(dates[0]), parse_date(dates[-1]), freq='D')
    
    # Create dataframe, set index and empty columns
    return pd.DataFrame(index=index, columns=column_array)
    
def set_dataframe_value(dataframe,entry,column):
    """ Inserts data into dataframe """
    # Obtain date for entry
    date=parse_date(entry['Date'])

    # Check if no data already exists for a date - prevent possible duplication if same date exists
    if pd.isna(dataframe.loc[date,column]):
        # Convert to float or replace None with 0 in our data 
        val = float(entry[column]) if entry[column]!= None else 0.0
        # Insert data into dataframe
        dataframe.loc[date,column]=val
    

In [28]:
def wrangle_data(rawdata, column_array):
    """ Creates, fills and returns a dataframe from rawdata and user specified columns
    Parameters: rawdata - data from json file or API call. 
    column_array - columns to be included in dataframe. """
    
    # Create initial dataframe skeleton
    df=initialize_dataframe(rawdata, column_array)

    # Iterate through each entry of rawdata 
    for entry in rawdata:
        # Iterate through all columns that were passed as a parameter
        for column in column_array:
            # Insert values into dataframe
            set_dataframe_value(df,entry,column)
        
    # fill in any remaining "holes" due to missing dates
    df.fillna(0.0, inplace=True)
    
    return df

# Creation and population of dataframes from rawdata:
hospital_ventilator_df=wrangle_data(covid_jsondata, ['Hospital Admissions','Occupied Ventilator Beds','Daily Deaths']) 
vaccine_cases_df=wrangle_data(covid_jsondata, ['Daily Cases','Daily Vaccines Administered'])

In [29]:
# Initialize the filters and structure necessary to make API call and obtain latest data

filters = [
    'areaType=overview'
]

structure = {
        "Date": "date",
        "Hospital Admissions": "newAdmissions",
        "Occupied Ventilator Beds": "covidOccupiedMVBeds",
        "Daily Deaths": "newDailyNsoDeathsByDeathDate",
        "Daily Vaccines Administered": "newVaccinesGivenByPublishDate",
        "Daily Cases": "newCasesByPublishDate"
}
      

def access_api():
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """
    api = Cov19API(filters=filters, structure=structure)
    return api.get_json()
        

## Refresh button
Clicking on the REFRESH button below will download fresh raw data from the [uk covid 19 API](https://coronavirus.data.gov.uk/details/developers-guide/main-api), and update existing dataframes and graphs to show the latest statistics.

The button will also change appearance to give user feedback if the process is successful or unsuccessful. 
- WARNING: It might take a few seconds before a response is obtained from the API and the button is updated.

In [30]:
def api_button_callback(button):
    """ Button callback function - button parameter is necessary but unused.
    Callback accesses API, wrangles data and updates both global variable 
    dataframes used for plotting graphs """
    # try - catch used for error handling - in case servers are down or other issues arise with getting API data

    try:
        # Get updated data from API
        apidata=access_api()['data']
      
        # wrangle the data and overwrite the existing dataframes
        global hospital_ventilator_df
        global vaccine_cases_df
        
        hospital_ventilator_df=wrangle_data(apidata, ['Hospital Admissions','Occupied Ventilator Beds','Daily Deaths'])
        vaccine_cases_df=wrangle_data(apidata, ['Daily Cases','Daily Vaccines Administered'])
        
        # Update button to show success
        apibutton.icon="check"
        apibutton.disabled=True
        apibutton.description="REFRESHED"

    except:
        # Disable button and give user negative feedback if API call fails
        apibutton.disabled=True
        apibutton.icon="times"
        apibutton.description="UNAVAILABLE"
        
    # Refresh graph with updated values
    refresh_graph()

# Create button used to update data
apibutton=wdg.Button(
    description='REFRESH',
    disabled=False,
    button_style='info', 
    tooltip="Update covid19 data",
    icon='download'
)

# Callback for button click
apibutton.on_click(api_button_callback) 

# Display button
display(apibutton)

Button(button_style='info', description='REFRESH', icon='download', style=ButtonStyle(), tooltip='Update covid…

In [31]:
# Widget initialization for graphs

# Create widget that allows viewing of multiple statistics
hospital_ventilator_options=wdg.SelectMultiple(
    options=['Hospital Admissions','Occupied Ventilator Beds','Daily Deaths'],
    value=['Hospital Admissions','Occupied Ventilator Beds','Daily Deaths'],
    rows=3,
    description='Stats:',
    disabled=False
)

# Initialize range of dates for timeframe widget
dates = pd.date_range(
    hospital_ventilator_df.index[0], 
    hospital_ventilator_df.index[-1], 
    freq='D')
options = [(date.strftime(' %b %Y '), date) for date in dates]

# Create widget that allows user to select range of dates
hospital_ventilator_timeframe=wdg.SelectionRangeSlider(
    options=options,
    index=(0, len(options)-1),
    description='Time',
    disabled=False,
    layout={'width':'500px'}
)

# Create widget that allows user to toggle between linear and logarithmic view
vaccine_cases_scale=wdg.RadioButtons(
    options=['Linear', 'Logarithmic'],
    description='Scale:',
    disabled=False
)

In [32]:
# Plotting graphs

def plot_ventilator_hospital(option, timeframe):
    hospital_ventilator_df.loc[timeframe[0] : timeframe[1]][list(option)].plot()
    plt.show()
    
def plot_vaccine_cases(scale):
    # Toggles between linear and logarithmic views
    if scale=='Linear':
        logscale=False
    else:
        logscale=True
    vaccine_cases_df.plot(logy=logscale)
    plt.show()

def refresh_graph():
    """ Change widget values to redraw hospital_ventilator graph """
    current=hospital_ventilator_options.value
    if current==hospital_ventilator_options.options[0:3]:
        other=hospital_ventilator_options.options[0:2]
    else:
        other=hospital_ventilator_options.options[0:3]

    # Swap values to force graph redraw
    hospital_ventilator_options.value=other 
    hospital_ventilator_options.value=current

# Create horizontal box for widgets
ventilator_hospital_controls=wdg.HBox([hospital_ventilator_options, hospital_ventilator_timeframe])

# plot graphs and add widgets to them
ventilator_hospital_graph=wdg.interactive_output(
    plot_ventilator_hospital, 
    {'option': hospital_ventilator_options, 'timeframe':hospital_ventilator_timeframe})

vaccine_cases_graph=wdg.interactive_output(plot_vaccine_cases, {'scale': vaccine_cases_scale})

## Occupied ventilator beds, new hospital admissions and deaths

This graph compares the daily number of hospital admissions, to occupied mechanical ventilator beds, and daily deaths.

Using the widgets you can:
- Select multiple different statistics
- Change the time frame for the statistics using the slider


In [33]:
display(ventilator_hospital_controls, ventilator_hospital_graph)

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': '<Figure size 1000x500 with 1 Axes>', '…

## Daily vaccine and case numbers

This graph compares the daily number of vaccines administered to daily new covid cases.

Using the widget you can toggle between seeing a linear or logarithmic view for the statistics.

In [34]:
display(vaccine_cases_scale, vaccine_cases_graph)

RadioButtons(description='Scale:', options=('Linear', 'Logarithmic'), value='Linear')

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': '<Figure size 1000x500 with 1 Axes>', '…